# This notebook test self encoding

In [2]:
import pandas as pd
import os
import datetime
from QuotesDownloader import SoftfxDownloader, QuotesType, QuotesPeriodicity
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses
from tensorflow.keras.utils import plot_model


Plan:
1. Download data for EURUSD
2. Convert data to pips
3. Convert bar prices to one pip movement. If bid was increased to X pips, X ones will be generated.
4. Create network for encoding/decoding to dictionary.


###  First step. Download M1 data.

In [3]:
symbols = ["EURUSD"]
pips_value = [10**-5]
N_x = 1000 #number of pips movements for NN

qd = SoftfxDownloader.Downloader()
train_raw = qd.get_quotes(symbols[0], datetime.date(2020, 1, 1), datetime.date(2021, 1, 1), QuotesPeriodicity.M1, QuotesType.Bids)
train_raw = train_raw.to_numpy()
print(train_raw.shape)
print(train_raw.dtype)

(368053, 6)
object


###  Second step. Convert data to pips

In [4]:
#Encode input data to NN friendly format.
bids = train_raw[:,1] / pips_value
bids = bids.astype('f')
bids = np.rint(bids).astype('i')
bids.shape

(368053,)

###  Fourth step. Convert to pip steps

In [ ]:
# initializing list
def pips2pricemovments(arr : np.ndarray) -> np.ndarray:
    d_iter = map(lambda x, y: np.repeat(1, y-x) if x<y else np.repeat(0, x-y), arr[:-1], arr[1:])
    return np.concatenate(list(d_iter))



In [6]:
filename_converted = "bids_EURUSD.npy"
if os.path.exists(filename_converted):
    print("Loading file")
    d = np.load(filename_converted)
else:
    d = pips2pricemovments(bids)
    np.save(filename_converted, d)
d.shape

Loading file


(3672304,)

##  Fifth step. NN

In [7]:
from dataclasses import dataclass
#train/test split
# Test your function and save all "global" variables within the G class (G stands for global)
@dataclass
class GSettings:
    filename_converted = "bids_EURUSD.npy"
    d = np.load(filename_converted)
    N = d.shape[0]
    train_ratio = 0.1
    n_train = int(N * train_ratio)
    window_size = 1024
    batch_size = 64
    SHUFFLE_BUFFER_SIZE = 1000
    N_letters = 10

train_data = d[:GSettings.n_train]
test_data = d[GSettings.n_train:]

print(f"Train data shape is {train_data.shape}.\nTest data shape is {test_data.shape}.")

Train data shape is (367230,).
Test data shape is (3305074,).


In [8]:
import tensorflow as tf
def windowed_dataset(series, window_size=GSettings.window_size, batch_size=GSettings.batch_size, shuffle_buffer=GSettings.SHUFFLE_BUFFER_SIZE):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w, w))
    ds = ds.batch(batch_size)#.prefetch(1)
    #ds = ds.map(lambda x, y: (x, tf.squeeze(y, axis=-1)))
    #for window in ds:
     #  print(list(window))
    return ds

train_set = windowed_dataset(train_data)
test_set = windowed_dataset(test_data)

#t = train_set.take(1)
#list(t.as_numpy_iterator())

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
class Alphabet(Model):
    def __init__(self):
        super(Alphabet, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(GSettings.window_size, 1)),
            layers.Conv1D(2, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(4, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(8, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(16, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(32, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Flatten()
            ,layers.Dense(54)
        ])
        latent_shape = self.encoder.layers[-2].input_shape
        print(latent_shape)
        self.decoder = tf.keras.Sequential([
            layers.Input(shape=54),
            layers.Dense(latent_shape[1]*latent_shape[2], activation='relu'),
            layers.Reshape(latent_shape[1:]),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(32, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(16, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(8, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(4, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(2, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.Conv1D(1, 3, activation = 'sigmoid', padding = 'same')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


In [10]:


class AlphabetMulti(Model):
    def __init__(self):
        super(AlphabetMulti, self).__init__()
        self.nne1 = tf.keras.Sequential([
            layers.Input(shape=(GSettings.window_size)),
            layers.Conv1D(2, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(4, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(8, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(16, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid'),
            layers.Conv1D(32, 9, activation='relu', padding='same', strides=1),
            layers.MaxPool1D(2, strides=2, padding='valid')
        ])
        nne1_outputshape = self.nne1.layers[-1].output_shape
        print(nne1_outputshape)

        self.encoder1 = tf.keras.Sequential([
            layers.Input(shape=nne1_outputshape[1:]),
            layers.Flatten(),
            layers.Dense(100, activation='sigmoid')
        ])

        # self.nne2 = tf.keras.Sequential([
        #     layers.Input(shape=nne1_outputshape[1:]),
        #     layers.Conv1D(32, 9, activation='relu', padding='same', strides=1),
        #     layers.MaxPool1D(2, strides=2, padding='valid'),
        #     layers.Conv1D(64, 9, activation='relu', padding='same', strides=1),
        #     layers.MaxPool1D(2, strides=2, padding='valid')
        # ])
        #
        # nne2_outputshape = self.nne2.layers[-1].output_shape
        # print(nne2_outputshape)
        #
        # self.encoder2 = tf.keras.Sequential([
        #     layers.Input(shape=nne2_outputshape[1:]),
        #     layers.Flatten(),
        #     layers.Dense(1024, activation='relu'),
        #     layers.Dense(100, activation='relu'),
        #     layers.Dense(10, activation='sigmoid')
        # ])

        self.decoder1 = tf.keras.Sequential([
            layers.Input(shape=100),
            layers.Dense(nne1_outputshape[1]*nne1_outputshape[2], activation='relu'),
            layers.Reshape(nne1_outputshape[1:])
        ])
        self.nnd1 = tf.keras.Sequential([
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(32, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(16, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(8, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(4, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.UpSampling1D(2),
            layers.Conv1DTranspose(2, kernel_size=9, strides=1, activation='relu', padding='same'),
            layers.Conv1D(1, 3, activation = 'sigmoid', padding = 'same', name = 'nnd1')
        ])
        # self.nnd1_ = tf.keras.Sequential([
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(32, kernel_size=9, strides=1, activation='relu', padding='same'),
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(16, kernel_size=9, strides=1, activation='relu', padding='same'),
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(8, kernel_size=9, strides=1, activation='relu', padding='same'),
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(4, kernel_size=9, strides=1, activation='relu', padding='same'),
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(2, kernel_size=9, strides=1, activation='relu', padding='same'),
        #     layers.Conv1D(1, 3, activation = 'sigmoid', padding = 'same', name = 'nnd1_')
        # ])
        # self.decoder2 = tf.keras.Sequential([
        #     layers.Input(shape=10),
        #     layers.Dense(100, activation='relu'),
        #     layers.Dense(1024, activation='relu'),
        #     layers.Dense(nne2_outputshape[1]*nne2_outputshape[2], activation='relu'),
        #     layers.Reshape(nne2_outputshape[1:])
        # ])
        # self.nnd2 = tf.keras.Sequential([
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(64, kernel_size=9, strides=1, activation='relu', padding='same'),
        #     layers.UpSampling1D(2),
        #     layers.Conv1DTranspose(32, kernel_size=9, strides=1, activation='relu', padding='same')
        # ])

    def call(self, x):
        nne1_result = self.nne1(x)
        encoded1 = self.encoder1(nne1_result)
        decoded1 = self.decoder1(encoded1)
        decoded1 = self.nnd1(decoded1)

        #nne2_result = self.nne2(nne1_result)
        #encoded2 = self.encoder2(nne2_result)

        #decoded2 = self.decoder2(encoded2)
        #decoded2 = self.nnd2(decoded2)
        #decoded2 = self.nnd1_(decoded2)

        return [decoded1]


In [11]:

class AlphabetTest(Model):
    def __init__(self):
        super(AlphabetTest, self).__init__()
        self.nne1 = tf.keras.Sequential([
            layers.Input(shape=(GSettings.window_size)),
            layers.Flatten(),
            layers.Dense(100, activation='relu')
        ])
        self.encoder1 = tf.keras.Sequential([
            layers.Dense(100, activation='sigmoid')
        ])
        self.decoder1 = tf.keras.Sequential([
            layers.Input(shape=100),
            layers.Dense(100, activation='relu')
        ])
        self.nnd1 = tf.keras.Sequential([
            layers.Dense(GSettings.window_size, activation='relu'),
            layers.Reshape((GSettings.window_size, 1))
        ])

    def call(self, x):
        nne1_result = self.nne1(x)
        encoded1 = self.encoder1(nne1_result)
        decoded1 = self.decoder1(encoded1)
        decoded1 = self.nnd1(decoded1)

        #nne2_result = self.nne2(nne1_result)
        #encoded2 = self.encoder2(nne2_result)

        #decoded2 = self.decoder2(encoded2)
        #decoded2 = self.nnd2(decoded2)
        #decoded2 = self.nnd1_(decoded2)

        return decoded1


In [12]:
autoencoder = AlphabetTest() #AlphabetSimple()
#autoencoder.build((GSettings.window_size))
#plot_model(autoencoder.nne1, show_shapes=True, show_layer_names=True)
#autoencoder.encoder1.summary()
#autoencoder.decoder1.summary()

In [13]:
# td = np.random.randint(0,2,GSettings.window_size)
# td = np.reshape(td, (1, 540, 1))
# print(td.shape)
# autoencoder.encoder(td)

In [14]:
# td = np.zeros(54)
# td[0] = 1
# td = td[np.newaxis, :]
# autoencoder.decoder(td)

In [15]:
#autoencoder.nne1.summary()

In [16]:
import sys
class CustomMSE(tf.keras.losses.Loss):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, y_true, y_pred):
        #print(y_true.shape)
        #mse1 = tf.reduce_mean(tf.square(y_true[:, 0] - y_pred[:, 0]))
        #mse2 = tf.reduce_mean(tf.square(y_true[:, 1] - y_pred[:, 1]))
        #total_mse = mse1 + mse2
        #return total_mse
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        return tf.reduce_mean(tf.square(y_true - y_pred))

def custom_loss_function(y_true, y_pred):
    tf.print("\n y_true", y_true, output_stream=sys.stdout)
    tf.print("\n y_pred", y_pred, output_stream=sys.stdout)

   # y_true = tf.cast(y_true, tf.float32)
    #y_pred = tf.cast(y_pred, tf.float32)


    #squared_difference = tf.square(y_true - y_pred)
    #return tf.reduce_mean(squared_difference)
    return tf.square(y_pred)


autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy']) #'binary_crossentropy'


In [25]:
#from tensorflow.keras.losses import mean_squared_error
autoencoder.compile(loss = mean_squared_error(param=2))

TypeError: Got an unexpected keyword argument 'param'

In [90]:
history = autoencoder.fit(train_set, epochs=1)

    145/Unknown - 11s 68ms/step - loss: 0.3125 - accuracy: 0.5456

KeyboardInterrupt: 

In [ ]:
y_true = [0, 1, 0, 0]
y_pred = [-18.6, 0.51, 2.94, -12.8]
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(y_true, y_pred).numpy()
